In [34]:
from snowflake.snowpark.session import Session
import json
connection_parameters = json.load(open('connection.json'))
session = Session.builder.configs(connection_parameters).create()
session

d:\College\7374\FinalProject\env\lib\site-packages\snowflake\connector\options.py:107: UserWarning: You have an incompatible version of 'pyarrow' installed (11.0.0), please install a version that adheres to: 'pyarrow<10.1.0,>=10.0.1; extra == "pandas"'
  warn_incompatible_dep(
Failed to import ArrowResult. No Apache Arrow result set format can be used. ImportError: DLL load failed while importing arrow_iterator: The specified procedure could not be found.


In [35]:
session

In [ ]:

# session.sql('create or replace stage dash_udfs').collect()
session.file.put("../temp/download.png",'@dash_models',overwrite=True,auto_compress=False)

[PutResult(source='download.png', target='download.png', source_size=4111, target_size=4112, source_compression='NONE', target_compression='NONE', status='UPLOADED', message='')]

In [3]:
# import pandas as pd
# df = pd.DataFrame({"FILE_NAME": ["image1"], "OUTPUT": ["cat"]})
# s = session.sql("session.sql('create database if not exists IMAGES')")
# s
# session.write_pandas(df, "IMAGES")

In [17]:
session.file.put('../temp/image_1.jpeg','@dash_models',overwrite=True,auto_compress=False)

[PutResult(source='image_1.jpeg', target='image_1.jpeg', source_size=26237, target_size=26240, source_compression='NONE', target_compression='NONE', status='UPLOADED', message='')]

In [18]:
import os
# Model Directory
directory = '../model/'
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        # Import trained model
        # session.file.put(f,'@dash_models',overwrite=True,auto_compress=False)
        session.add_import('@dash_models/'+filename)

In [38]:
session.file.put("../backend/images/image.jpg",'@dash_models',overwrite=True,auto_compress=False)


[PutResult(source='image.jpg', target='image.jpg', source_size=85949, target_size=85952, source_compression='NONE', target_compression='NONE', status='UPLOADED', message='')]

In [39]:
from snowflake.snowpark.functions import udf, sum, col,array_construct,month,year,call_udf,lit
import os
from snowflake.snowpark.types import  StringType, IntegerType

# session.add_import('@dash_models/m/tokenizer.json')
# session.add_import('@dash_models/pytorch_model.bin')
# session.add_import('@dash_files/mobilenetv3.py')
session.add_import('@dash_models/image.jpg')
@udf(name='image_caption_generator',session=session,replace=True,is_permanent=True,stage_location='@dash_models')
def image_caption_generator() -> str:
  from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, AutoTokenizer
  import torch
  import sys
  from PIL import Image
  from io import BytesIO
  import requests
  import glob 
  IMPORT_DIRECTORY_NAME = "snowflake_import_directory"
  import_dir = sys._xoptions[IMPORT_DIRECTORY_NAME]
  # return os.listdir(import_dir)
  
  model_path = import_dir

  model = VisionEncoderDecoderModel.from_pretrained(model_path)
  feature_extractor = ViTFeatureExtractor.from_pretrained(model_path)
  tokenizer = AutoTokenizer.from_pretrained(model_path)
  image_file_name = "/image.jpg"
  device = torch.device("cpu" if torch.cuda.is_available() else "cpu")
  model.to(device)
  max_length = 16
  num_beams = 4
  gen_kwargs = {"max_length": max_length, "num_beams": num_beams}
  images = []
  i_image = Image.open(model_path+image_file_name)
  if i_image.mode != "RGB":
    i_image = i_image.convert(mode="RGB")

  images.append(i_image)

  pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
  pixel_values = pixel_values.to(device)

  output_ids = model.generate(pixel_values, **gen_kwargs)

  preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
  preds = [pred.strip() for pred in preds]
  # os.remove(model_path+"downloa")
  return preds[0]

In [ ]:
def generate_raw_caption(image_link : str) -> str:
    # Snowpark imports 
    from snowflake.snowpark.session import Session
    import json
    connection_parameters = json.load(open('connection.json'))
    session = Session.builder.configs(connection_parameters).create()
    # ... continued in backend

In [40]:
# image_links_in_str = ["https://as2.ftcdn.net/v2/jpg/03/03/62/45/1000_F_303624505_u0bFT1Rnoj8CMUSs8wMCwoKlnWlh5Jiq.jpg"]
predicted_label = session.sql('''SELECT image_caption_generator()''').collect()


In [41]:
predicted_label[0][0]

'a wooden table topped with a wooden table cloth'

In [4]:
session.add_packages(["transformers","Pillow"])

The version of package transformers in the local environment is 4.28.1, which does not fit the criteria for the requirement transformers. Your UDF might not work when the package version is different between the server and your local environment
The version of package pillow in the local environment is 9.5.0, which does not fit the criteria for the requirement pillow. Your UDF might not work when the package version is different between the server and your local environment


In [12]:
session.clear_imports()
session.clear_packages()

In [18]:
import requests
url = 'https://4t6zjqbvra77cljo42ody4xtmi0smplq.lambda-url.us-east-1.on.aws/'
data = {'method': 'LOGIN_USER', 'userpk': 'user#'+"username1"}
res = requests.post(url, json=data)
res = res.json()
print(res)

{'createdAt': '2023-04-25 19:54:28', 'sk': 'user#username1', 'pk': 'user#username1', 'interests': []}


In [1]:
import requests
url = 'https://4t6zjqbvra77cljo42ody4xtmi0smplq.lambda-url.us-east-1.on.aws/'
data = {'method': 'LIST_POSTS'}
res = requests.post(url, json=data)
res = res.json()
print(res)

[{'image_link': 'https://admmedia.s3.ap-south-1.amazonaws.com/user_uploads/img_0390164a-4946-469e-ae1a-b4cbff3e8dbb', 'sk': 'img_0390164a-4946-469e-ae1a-b4cbff3e8dbb', 'pk': 'post', 'recognitions': []}, {'image_link': 'https://admmedia.s3.ap-south-1.amazonaws.com/user_uploads/img_581c53dd-7c99-4ce0-b737-c34987ff8550', 'sk': 'img_581c53dd-7c99-4ce0-b737-c34987ff8550', 'pk': 'post', 'recognitions': []}, {'generated_interested': ['cat', 'guitar'], 'image_link': 'https://admmedia.s3.ap-south-1.amazonaws.com/user_uploads/exsample.png', 'sk': 'post#2', 'pk': 'post'}, {'generated_interested': ['cat', 'guitar'], 'image_link': 'https://admmedia.s3.ap-south-1.amazonaws.com/user_uploads/exsample.png', 'sk': 'post#3', 'pk': 'post'}]


In [15]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
# Function to download and save image locally

nltk.data.path.append('../nltk_data')

In [16]:
def extract_nouns(caption):
    text = caption
    words = word_tokenize(text)
    tagged_words = pos_tag(words)
    nouns = [word for word, tag in tagged_words if tag in ['NN', 'NNS']]
    return nouns, caption
extract_nouns("quick brown fox was running")

(['brown', 'fox'], 'quick brown fox was running')

In [14]:
import requests
from bs4 import BeautifulSoup

# Function to fetch product details
def fetch_product_details(product_name):
    # URL of the Amazon search page for the given product
    url = f"https://www.amazon.com/s?k={product_name}&ref=nb_sb_noss_1"
    headers = ({'User-Agent':
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})

    # Send an HTTP GET request to the Amazon search page
    response = requests.get(url,headers=headers)
    print(response)
    # Parse the HTML content of the Amazon search page
    soup = BeautifulSoup(response.content, "html.parser")

    # Find the first 3 product links on the search page
    product_links = soup.find_all("a", {"class": "a-section"})
    print(product_links)
    # Fetch the product details for each product link
    for link in product_links[:3]:
        # Extract the product title
        title = link.find("span", {"class": "a-size-medium a-color-base a-text-normal"}).text.strip()
        print(title)
        # Extract the product price
        price = link.find("span", {"class": "a-price-whole"})
        if price is None:
            price = link.find("span", {"class": "a-offscreen"})
        if price is not None:
            price = price.text.strip()

        # Extract the product image URL
        image_url = link.find("img")["src"]

        # Extract the product link
        product_url = "https://www.amazon.com" + link["href"]

        # Print the product details
        print(f"Title: {title}")
        print(f"Price: {price}")
        print(f"Image URL: {image_url}")
        print(f"Product URL: {product_url}")
        print("\n")

# Example usage
fetch_product_details("laptop")


<Response [200]>
[]


In [33]:
from bs4 import BeautifulSoup
import requests

# Function to extract Product Title
def get_title(soup):
	
	try:
		# Outer Tag Object
		title = soup.find("span", attrs={"id":'productTitle'})

		# Inner NavigatableString Object
		title_value = title.string

		# Title as a string value
		title_string = title_value.strip()

		# # Printing types of values for efficient understanding
		# print(type(title))
		# print(type(title_value))
		# print(type(title_string))
		# print()

	except AttributeError:
		title_string = ""	

	return title_string

# Function to extract Product Price
def get_price(soup):

	try:
		price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

	except AttributeError:

		try:
			# If there is some deal price
			price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

		except:		
			price = ""	

	return price

# Function to extract Product Rating
def get_rating(soup):

	try:
		rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
		
	except AttributeError:
		
		try:
			rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
		except:
			rating = ""	

	return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
	try:
		image_element = soup.find('img', {'id': 'landingImage'})
		image_url = image_element['data-old-hires']
		print(f"Product Image URL: {image_url}")
		return image_url
	except AttributeError:
		image_url = ""	

	return image_url

# Function to extract Availability Status
def get_availability(soup):
	try:
		available = soup.find("div", attrs={'id':'availability'})
		available = available.find("span").string.strip()

	except AttributeError:
		available = "Not Available"	

	return available	


def get_ads(product_name):

	# Headers for request
	HEADERS = ({'User-Agent':
	            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
	            'Accept-Language': 'en-US'})

	# The webpage URL
	URL = f"https://www.amazon.com/s?k={product_name}&ref=nb_sb_noss_2"
	
	# HTTP Request
	webpage = requests.get(URL, headers=HEADERS)
	print(webpage)
	# Soup Object containing all data
	soup = BeautifulSoup(webpage.content, "html.parser")

	# Fetch links as List of Tag Objects
	links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})[:3]
	print(links)
	# Store the links
	links_list = []

	# Loop for extracting links from Tag Objects
	for link in links:
		print(link)
		links_list.append(link.get('href'))

	ads = []
	# Loop for extracting product details from each link 
	for link in links_list:

		new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

		new_soup = BeautifulSoup(new_webpage.content, "html.parser")
		
		# Function calls to display all necessary product information
		ads.append({
			"title": get_title(new_soup),
			"image_url": get_review_count(new_soup),
			"rating": get_rating(new_soup),
			"link": "https://www.amazon.com"+ link
		})
	return ads
get_ads("laptop")


<Response [503]>
[]


[]

In [3]:
import requests
api_url = 'https://4t6zjqbvra77cljo42ody4xtmi0smplq.lambda-url.us-east-1.on.aws/'

data = {'method': 'POST_RECOGNITION', 'postsk':'img_326e9c11-4936-4ba3-b4e0-47cf0da771eb.jpg', 'caption':"tssssssss", "ad_links": ["s","a","a"], "recommends":["k","e","t","h"]}
res = requests.post(api_url, json=data)
res    

<Response [502]>

In [ ]:
elif (event['method'] == "POST_RECOGNITION"):
        bucket_name = 'admmedia'
        s3_client = boto3.client('s3')
        key = {'pk': {'S': 'post'}, 'sk': {'S': event['postsk']}}

        # specify the attributes to update
        update_expression = 'SET #attr1 = :rec, #attr2 = :cap, #attr3 = :ad_links '
        expression_attribute_names = {'#attr1': 'recommends', '#attr2': 'caption' , '#attr3': 'ad_links' }
        expression_attribute_values = {':rec': {'S': event["recommends"]}, ':cap': {'N': event["caption"]}, ':links': {'N': event["ad_links"]}}
        
        # update the item in the table
        response = dynamodb.update_item(
            TableName=table_name,
            Key=key,
            UpdateExpression=update_expression,
            ExpressionAttributeNames=expression_attribute_names,
            ExpressionAttributeValues=expression_attribute_values,
        )
        return {
            'statusCode': response['ResponseMetadata']['HTTPStatusCode'],
            'body': "post_updated",
                "headers": {
            'Content-Type': 'text/html',
        }
        }